Thanks to Taylor S. Amarel for the code this submission is based off,

What changed from the versions.
increased n-estimstors and the learning rate:
n_estimators: 150 -> 200
learning rate: 0.03 -> 0.06

In [ ]:
# MABe Challenge - Social Action Recognition in Mice
# Complete code with long-range temporal dependencies

validate_or_submit = 'submit' # 'validate' or 'submit' or 'stresstest'
verbose = True

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange, tqdm
import itertools
import warnings
import json
import os
import lightgbm
from collections import defaultdict
import polars as pl

from sklearn.base import ClassifierMixin, BaseEstimator, clone
from sklearn.model_selection import cross_val_predict, GroupKFold, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

# Custom classifier for training on subset
class TrainOnSubsetClassifier(ClassifierMixin, BaseEstimator):
    """Fit estimator to a subset of the training data."""
    def __init__(self, estimator, n_samples):
        self.estimator = estimator
        self.n_samples = n_samples

    def fit(self, X, y):
        downsample = len(X) // self.n_samples
        downsample = max(downsample, 1)
        self.estimator.fit(np.array(X, copy=False)[::downsample],
                           np.array(y, copy=False)[::downsample])
        self.classes_ = self.estimator.classes_
        return self

    def predict_proba(self, X):
        if len(self.classes_) == 1:
            return np.full((len(X), 1), 1.0)
        probs = self.estimator.predict_proba(np.array(X))
        return probs
        
    def predict(self, X):
        return self.estimator.predict(np.array(X))

# F-Beta scoring functions
class HostVisibleError(Exception):
    pass

def single_lab_f1(lab_solution: pl.DataFrame, lab_submission: pl.DataFrame, beta: float = 1) -> float:
    label_frames: defaultdict[str, set[int]] = defaultdict(set)
    prediction_frames: defaultdict[str, set[int]] = defaultdict(set)

    for row in lab_solution.to_dicts():
        label_frames[row['label_key']].update(range(row['start_frame'], row['stop_frame']))

    for video in lab_solution['video_id'].unique():
        active_labels: str = lab_solution.filter(pl.col('video_id') == video)['behaviors_labeled'].first()
        active_labels: set[str] = set(json.loads(active_labels))
        predicted_mouse_pairs: defaultdict[str, set[int]] = defaultdict(set)

        for row in lab_submission.filter(pl.col('video_id') == video).to_dicts():
            if ','.join([str(row['agent_id']), str(row['target_id']), row['action']]) not in active_labels:
                continue
           
            new_frames = set(range(row['start_frame'], row['stop_frame']))
            new_frames = new_frames.difference(prediction_frames[row['prediction_key']])
            prediction_pair = ','.join([str(row['agent_id']), str(row['target_id'])])
            if predicted_mouse_pairs[prediction_pair].intersection(new_frames):
                raise HostVisibleError('Multiple predictions for the same frame from one agent/target pair')
            prediction_frames[row['prediction_key']].update(new_frames)
            predicted_mouse_pairs[prediction_pair].update(new_frames)

    tps = defaultdict(int)
    fns = defaultdict(int)
    fps = defaultdict(int)
    for key, pred_frames in prediction_frames.items():
        action = key.split('_')[-1]
        matched_label_frames = label_frames[key]
        tps[action] += len(pred_frames.intersection(matched_label_frames))
        fns[action] += len(matched_label_frames.difference(pred_frames))
        fps[action] += len(pred_frames.difference(matched_label_frames))

    distinct_actions = set()
    for key, frames in label_frames.items():
        action = key.split('_')[-1]
        distinct_actions.add(action)
        if key not in prediction_frames:
            fns[action] += len(frames)

    action_f1s = []
    for action in distinct_actions:
        if tps[action] + fns[action] + fps[action] == 0:
            action_f1s.append(0)
        else:
            action_f1s.append((1 + beta**2) * tps[action] / ((1 + beta**2) * tps[action] + beta**2 * fns[action] + fps[action]))
    return sum(action_f1s) / len(action_f1s)

def mouse_fbeta(solution: pd.DataFrame, submission: pd.DataFrame, beta: float = 1) -> float:
    if len(solution) == 0 or len(submission) == 0:
        raise ValueError('Missing solution or submission data')

    expected_cols = ['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame']

    for col in expected_cols:
        if col not in solution.columns:
            raise ValueError(f'Solution is missing column {col}')
        if col not in submission.columns:
            raise ValueError(f'Submission is missing column {col}')

    solution: pl.DataFrame = pl.DataFrame(solution)
    submission: pl.DataFrame = pl.DataFrame(submission)
    assert (solution['start_frame'] <= solution['stop_frame']).all()
    assert (submission['start_frame'] <= submission['stop_frame']).all()
    solution_videos = set(solution['video_id'].unique())
    submission = submission.filter(pl.col('video_id').is_in(solution_videos))

    solution = solution.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('label_key'),
    )
    submission = submission.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('prediction_key'),
    )

    lab_scores = []
    for lab in solution['lab_id'].unique():
        lab_solution = solution.filter(pl.col('lab_id') == lab).clone()
        lab_videos = set(lab_solution['video_id'].unique())
        lab_submission = submission.filter(pl.col('video_id').is_in(lab_videos)).clone()
        lab_scores.append(single_lab_f1(lab_solution, lab_submission, beta=beta))

    return sum(lab_scores) / len(lab_scores)

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, beta: float = 1) -> float:
    solution = solution.drop(row_id_column_name, axis='columns', errors='ignore')
    submission = submission.drop(row_id_column_name, axis='columns', errors='ignore')
    return mouse_fbeta(solution, submission, beta=beta)

# Load data
train = pd.read_csv('/kaggle/input/MABe-mouse-behavior-detection/train.csv')
train['n_mice'] = 4 - train[['mouse1_strain', 'mouse2_strain', 'mouse3_strain', 'mouse4_strain']].isna().sum(axis=1)
train_without_mabe22 = train.query("~ lab_id.str.startswith('MABe22_')")

test = pd.read_csv('/kaggle/input/MABe-mouse-behavior-detection/test.csv')
body_parts_tracked_list = list(np.unique(train.body_parts_tracked))

# Solution dataframe creation function for validation
def create_solution_df(dataset):
    solution = []
    for _, row in tqdm(dataset.iterrows(), total=len(dataset)):
        lab_id = row['lab_id']
        if lab_id.startswith('MABe22'): continue
        video_id = row['video_id']
        path = f"/kaggle/input/MABe-mouse-behavior-detection/train_annotation/{lab_id}/{video_id}.parquet"
        try:
            annot = pd.read_parquet(path)
        except FileNotFoundError:
            if verbose: print(f"No annotations for {path}")
            continue
    
        annot['lab_id'] = lab_id
        annot['video_id'] = video_id
        annot['behaviors_labeled'] = row['behaviors_labeled']
        annot['target_id'] = np.where(annot.target_id != annot.agent_id, annot['target_id'].apply(lambda s: f"mouse{s}"), 'self')
        annot['agent_id'] = annot['agent_id'].apply(lambda s: f"mouse{s}")
        solution.append(annot)
    
    solution = pd.concat(solution)
    return solution

if validate_or_submit == 'validate':
    solution = create_solution_df(train_without_mabe22)

# Stress test code (only runs if validate_or_submit == 'stresstest')
if validate_or_submit == 'stresstest':
    n_videos_per_lab = 2
    
    try:
        os.mkdir(f"stresstest_tracking")
    except FileExistsError:
        pass
    
    rng = np.random.default_rng()
    stresstest = pd.concat(
        [train.query("video_id == 1459695188")]
        + [df.sample(min(n_videos_per_lab, len(df)), random_state=1) for (_, df) in train.groupby('lab_id')])
    for _, row in tqdm(stresstest.iterrows(), total=len(stresstest)):
        lab_id = row['lab_id']
        video_id = row['video_id']
        
        path = f"/kaggle/input/MABe-mouse-behavior-detection/train_tracking/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)
    
        if video_id == 1459695188:
            vid = pd.concat([vid] * 3)
            vid['video_frame'] = np.arange(len(vid))
    
        dropped_frames = list(rng.choice(np.unique(vid.video_frame), size=100, replace=False))
        vid = vid.query("~ video_frame.isin(@dropped_frames)")
        
        if rng.uniform() < 0.2:
            dropped_bodypart = rng.choice(np.unique(vid.bodypart), size=1, replace=False)[0]
            vid = vid.query("bodypart != @dropped_bodypart")
        
        if rng.uniform() < 0.1:
            vid = vid.query("mouse_id != 1")
        
        if rng.uniform() < 0.7:
            mask = np.ones(len(vid), dtype=bool)
            mask[:int(0.4 * len(mask))] = False
            rng.shuffle(mask)
            vid = vid[mask]
    
        if rng.uniform() < 0.7:
            mask = np.ones(len(vid), dtype=bool)
            mask[:int(0.2 * len(mask))] = False
            rng.shuffle(mask)
            vid.loc[:, 'x'] = np.where(mask, np.nan, vid.loc[:, 'x'])
            rng.shuffle(mask)
            vid.loc[:, 'y'] = np.where(mask, np.nan, vid.loc[:, 'y'])
    
        try:
            os.mkdir(f"stresstest_tracking/{lab_id}")
        except FileExistsError:
            pass
        new_path = f"stresstest_tracking/{lab_id}/{video_id}.parquet"
        vid.to_parquet(new_path)

# Body parts to drop for memory efficiency
drop_body_parts = ['headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright', 
                   'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright', 
                   'spine_1', 'spine_2',
                   'tail_middle_1', 'tail_middle_2', 'tail_midpoint']

# Generate mouse data function
def generate_mouse_data(dataset, traintest, traintest_directory=None, generate_single=True, generate_pair=True):
    assert traintest in ['train', 'test']
    if traintest_directory is None:
        traintest_directory = f"/kaggle/input/MABe-mouse-behavior-detection/{traintest}_tracking"
    for _, row in dataset.iterrows():
        
        lab_id = row.lab_id
        if lab_id.startswith('MABe22'): continue
        video_id = row.video_id

        if type(row.behaviors_labeled) != str:
            print('No labeled behaviors:', lab_id, video_id, type(row.behaviors_labeled), row.behaviors_labeled)
            continue

        path = f"{traintest_directory}/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)
        if len(np.unique(vid.bodypart)) > 5:
            vid = vid.query("~ bodypart.isin(@drop_body_parts)")
        pvid = vid.pivot(columns=['mouse_id', 'bodypart'], index='video_frame', values=['x', 'y'])
        if pvid.isna().any().any():
            if verbose and traintest == 'test': print('video with missing values', video_id, traintest, len(vid), 'frames')
        else:
            if verbose and traintest == 'test': print('video with all values', video_id, traintest, len(vid), 'frames')
        del vid
        pvid = pvid.reorder_levels([1, 2, 0], axis=1).T.sort_index().T
        pvid /= row.pix_per_cm_approx

        vid_behaviors = json.loads(row.behaviors_labeled)
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(',') for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])
        
        if traintest == 'train':
            try:
                annot = pd.read_parquet(path.replace('train_tracking', 'train_annotation'))
            except FileNotFoundError:
                continue

        if generate_single:
            vid_behaviors_subset = vid_behaviors.query("target == 'self'")
            for mouse_id_str in np.unique(vid_behaviors_subset.agent):
                try:
                    mouse_id = int(mouse_id_str[-1])
                    vid_agent_actions = np.unique(vid_behaviors_subset.query("agent == @mouse_id_str").action)
                    single_mouse = pvid.loc[:, mouse_id]
                    assert len(single_mouse) == len(pvid)
                    single_mouse_meta = pd.DataFrame({
                        'video_id': video_id,
                        'agent_id': mouse_id_str,
                        'target_id': 'self',
                        'video_frame': single_mouse.index
                    })
                    if traintest == 'train':
                        single_mouse_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=single_mouse.index)
                        annot_subset = annot.query("(agent_id == @mouse_id) & (target_id == @mouse_id)")
                        for i in range(len(annot_subset)):
                            annot_row = annot_subset.iloc[i]
                            single_mouse_label.loc[annot_row['start_frame']:annot_row['stop_frame'], annot_row.action] = 1.0
                        yield 'single', single_mouse, single_mouse_meta, single_mouse_label
                    else:
                        if verbose: print('- test single', video_id, mouse_id)
                        yield 'single', single_mouse, single_mouse_meta, vid_agent_actions
                except KeyError:
                    pass

        if generate_pair:
            vid_behaviors_subset = vid_behaviors.query("target != 'self'")
            if len(vid_behaviors_subset) > 0:
                for agent, target in itertools.permutations(np.unique(pvid.columns.get_level_values('mouse_id')), 2):
                    agent_str = f"mouse{agent}"
                    target_str = f"mouse{target}"
                    vid_agent_actions = np.unique(vid_behaviors_subset.query("(agent == @agent_str) & (target == @target_str)").action)
                    mouse_pair = pd.concat([pvid[agent], pvid[target]], axis=1, keys=['A', 'B'])
                    assert len(mouse_pair) == len(pvid)
                    mouse_pair_meta = pd.DataFrame({
                        'video_id': video_id,
                        'agent_id': agent_str,
                        'target_id': target_str,
                        'video_frame': mouse_pair.index
                    })
                    if traintest == 'train':
                        mouse_pair_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=mouse_pair.index)
                        annot_subset = annot.query("(agent_id == @agent) & (target_id == @target)")
                        for i in range(len(annot_subset)):
                            annot_row = annot_subset.iloc[i]
                            mouse_pair_label.loc[annot_row['start_frame']:annot_row['stop_frame'], annot_row.action] = 1.0
                        yield 'pair', mouse_pair, mouse_pair_meta, mouse_pair_label
                    else:
                        if verbose: print('- test pair', video_id, agent, target)
                        yield 'pair', mouse_pair, mouse_pair_meta, vid_agent_actions

# Threshold for multiclass prediction
threshold = 0.27

# Predict multiclass function
def predict_multiclass(pred, meta):
    ama = np.argmax(pred, axis=1)
    ama = np.where(pred.max(axis=1) >= threshold, ama, -1)
    ama = pd.Series(ama, index=meta.video_frame)
    changes_mask = (ama != ama.shift(1)).values
    ama_changes = ama[changes_mask]
    meta_changes = meta[changes_mask]
    mask = ama_changes.values >= 0
    mask[-1] = False
    submission_part = pd.DataFrame({
        'video_id': meta_changes['video_id'][mask].values,
        'agent_id': meta_changes['agent_id'][mask].values,
        'target_id': meta_changes['target_id'][mask].values,
        'action': pred.columns[ama_changes[mask].values],
        'start_frame': ama_changes.index[mask],
        'stop_frame': ama_changes.index[1:][mask[:-1]]
    })
    stop_video_id = meta_changes['video_id'][1:][mask[:-1]].values
    stop_agent_id = meta_changes['agent_id'][1:][mask[:-1]].values
    stop_target_id = meta_changes['target_id'][1:][mask[:-1]].values
    for i in range(len(submission_part)):
        video_id = submission_part.video_id.iloc[i]
        agent_id = submission_part.agent_id.iloc[i]
        target_id = submission_part.target_id.iloc[i]
        if i < len(stop_video_id):
            if stop_video_id[i] != video_id or stop_agent_id[i] != agent_id or stop_target_id[i] != target_id:
                new_stop_frame = meta.query("(video_id == @video_id)").video_frame.max() + 1
                submission_part.iat[i, submission_part.columns.get_loc('stop_frame')] = new_stop_frame
        else:
            new_stop_frame = meta.query("(video_id == @video_id)").video_frame.max() + 1
            submission_part.iat[i, submission_part.columns.get_loc('stop_frame')] = new_stop_frame
    assert (submission_part.stop_frame > submission_part.start_frame).all(), 'stop <= start'
    if verbose: print('  actions found:', len(submission_part))
    return submission_part

# Transform functions with LONG-RANGE TEMPORAL DEPENDENCIES
def transform_single(single_mouse, body_parts_tracked):
    available_body_parts = single_mouse.columns.get_level_values(0)
    X = pd.DataFrame({
            f"{part1}+{part2}": np.square(single_mouse[part1] - single_mouse[part2]).sum(axis=1, skipna=False)
            for part1, part2 in itertools.combinations(body_parts_tracked, 2) if part1 in available_body_parts and part2 in available_body_parts
        })
    X = X.reindex(columns=[f"{part1}+{part2}" for part1, part2 in itertools.combinations(body_parts_tracked, 2)], copy=False)

    if 'ear_left' in single_mouse.columns and 'ear_right' in single_mouse.columns and 'tail_base' in single_mouse.columns:
        shifted = single_mouse[['ear_left', 'ear_right', 'tail_base']].shift(10)
        X = pd.concat([
            X, 
            pd.DataFrame({
                'speed_left': np.square(single_mouse['ear_left'] - shifted['ear_left']).sum(axis=1, skipna=False),
                'speed_right': np.square(single_mouse['ear_right'] - shifted['ear_right']).sum(axis=1, skipna=False),
                'speed_left2': np.square(single_mouse['ear_left'] - shifted['tail_base']).sum(axis=1, skipna=False),
                'speed_right2': np.square(single_mouse['ear_right'] - shifted['tail_base']).sum(axis=1, skipna=False),
            })
        ], axis=1)
    
    # Add relative distances
    if 'nose+tail_base' in X.columns and 'ear_left+ear_right' in X.columns:
        X['elongation_ratio'] = X['nose+tail_base'] / (X['ear_left+ear_right'] + 1e-6)
    
    # Body angle/curvature
    if 'nose' in available_body_parts and 'body_center' in available_body_parts and 'tail_base' in available_body_parts:
        v1 = single_mouse['nose'] - single_mouse['body_center']
        v2 = single_mouse['tail_base'] - single_mouse['body_center']
        
        dot_product = (v1['x'] * v2['x'] + v1['y'] * v2['y'])
        norm_v1 = np.sqrt(v1['x']**2 + v1['y']**2)
        norm_v2 = np.sqrt(v2['x']**2 + v2['y']**2)
        
        X['body_angle_cos'] = dot_product / (norm_v1 * norm_v2 + 1e-6)
    
    # LONG-RANGE TEMPORAL DEPENDENCIES
    # Multiple time windows for capturing patterns at different scales
    time_windows = [5, 15, 30, 60]  # Short, medium, long, very long range
    
    if 'body_center' in available_body_parts:
        center_x = single_mouse['body_center']['x']
        center_y = single_mouse['body_center']['y']
        
        for window in time_windows:
            # Rolling statistics over different time windows
            X[f'center_x_mean_{window}'] = center_x.rolling(window, min_periods=1, center=True).mean()
            X[f'center_y_mean_{window}'] = center_y.rolling(window, min_periods=1, center=True).mean()
            X[f'center_x_std_{window}'] = center_x.rolling(window, min_periods=1, center=True).std()
            X[f'center_y_std_{window}'] = center_y.rolling(window, min_periods=1, center=True).std()
            
            # Movement range in window
            X[f'x_range_{window}'] = center_x.rolling(window, min_periods=1, center=True).max() - center_x.rolling(window, min_periods=1, center=True).min()
            X[f'y_range_{window}'] = center_y.rolling(window, min_periods=1, center=True).max() - center_y.rolling(window, min_periods=1, center=True).min()
            
            # Cumulative displacement over window
            X[f'cumulative_disp_{window}'] = np.sqrt(
                center_x.diff().rolling(window, min_periods=1).sum()**2 + 
                center_y.diff().rolling(window, min_periods=1).sum()**2
            )
            
            # Activity level (movement variance) over window
            X[f'activity_level_{window}'] = np.sqrt(
                center_x.diff().rolling(window, min_periods=1).var() + 
                center_y.diff().rolling(window, min_periods=1).var()
            )
    
    # Lag features at multiple time offsets
    if 'nose' in available_body_parts and 'tail_base' in available_body_parts:
        nose_tail_dist = np.sqrt(
            (single_mouse['nose']['x'] - single_mouse['tail_base']['x'])**2 + 
            (single_mouse['nose']['y'] - single_mouse['tail_base']['y'])**2
        )
        
        for lag in [10, 20, 40]:
            X[f'nose_tail_dist_lag_{lag}'] = nose_tail_dist.shift(lag)
            X[f'nose_tail_dist_diff_{lag}'] = nose_tail_dist - nose_tail_dist.shift(lag)
    
    # Temporal context features
    if 'ear_left' in available_body_parts and 'ear_right' in available_body_parts:
        ear_dist = np.sqrt(
            (single_mouse['ear_left']['x'] - single_mouse['ear_right']['x'])**2 + 
            (single_mouse['ear_left']['y'] - single_mouse['ear_right']['y'])**2
        )
        
        # Look both forward and backward in time
        for offset in [-20, -10, 10, 20]:
            X[f'ear_dist_offset_{offset}'] = ear_dist.shift(-offset)
        
        # Temporal consistency (how stable is the feature over time)
        X['ear_dist_consistency_30'] = ear_dist.rolling(30, min_periods=1, center=True).std() / (ear_dist.rolling(30, min_periods=1, center=True).mean() + 1e-6)
    
    return X#*1.04

def transform_pair(mouse_pair, body_parts_tracked):
    available_body_parts_A = mouse_pair['A'].columns.get_level_values(0)
    available_body_parts_B = mouse_pair['B'].columns.get_level_values(0)
    X = pd.DataFrame({
            f"12+{part1}+{part2}": np.square(mouse_pair['A'][part1] - mouse_pair['B'][part2]).sum(axis=1, skipna=False)
            for part1, part2 in itertools.product(body_parts_tracked, repeat=2) if part1 in available_body_parts_A and part2 in available_body_parts_B
        })
    X = X.reindex(columns=[f"12+{part1}+{part2}" for part1, part2 in itertools.product(body_parts_tracked, repeat=2)], copy=False)

    if ('A', 'ear_left') in mouse_pair.columns and ('B', 'ear_left') in mouse_pair.columns:
        shifted_A = mouse_pair['A']['ear_left'].shift(10)
        shifted_B = mouse_pair['B']['ear_left'].shift(10)
        X = pd.concat([
            X,
            pd.DataFrame({
                'speed_left_A': np.square(mouse_pair['A']['ear_left'] - shifted_A).sum(axis=1, skipna=False),
                'speed_left_AB': np.square(mouse_pair['A']['ear_left'] - shifted_B).sum(axis=1, skipna=False),
                'speed_left_B': np.square(mouse_pair['B']['ear_left'] - shifted_B).sum(axis=1, skipna=False),
            })
        ], axis=1)
    
    # Add relative distances
    if 'nose+tail_base' in X.columns and 'ear_left+ear_right' in X.columns:
        X['elongation_ratio'] = X['nose+tail_base'] / (X['ear_left+ear_right'] + 1e-6)
    
    # Relative orientation between mice
    if 'nose' in available_body_parts_A and 'tail_base' in available_body_parts_A and \
       'nose' in available_body_parts_B and 'tail_base' in available_body_parts_B:
        dir_A = mouse_pair['A']['nose'] - mouse_pair['A']['tail_base']
        dir_B = mouse_pair['B']['nose'] - mouse_pair['B']['tail_base']
        
        dot_product = (dir_A['x'] * dir_B['x'] + dir_A['y'] * dir_B['y'])
        norm_A = np.sqrt(dir_A['x']**2 + dir_A['y']**2)
        norm_B = np.sqrt(dir_B['x']**2 + dir_B['y']**2)
        
        X['relative_orientation_cos'] = dot_product / (norm_A * norm_B + 1e-6)
    
    # Approach rate
    if 'nose' in available_body_parts_A and 'nose' in available_body_parts_B:
        current_dist = np.square(mouse_pair['A']['nose'] - mouse_pair['B']['nose']).sum(axis=1, skipna=False)
        
        shifted_A_nose = mouse_pair['A']['nose'].shift(10)
        shifted_B_nose = mouse_pair['B']['nose'].shift(10)
        past_dist = np.square(shifted_A_nose - shifted_B_nose).sum(axis=1, skipna=False)
        
        X['approach_rate'] = current_dist - past_dist
    
    # Social zone indicators
    if 'body_center' in available_body_parts_A and 'body_center' in available_body_parts_B:
        center_dist = np.sqrt(
            (mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x'])**2 +
            (mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y'])**2
        )
        
        X['very_close'] = (center_dist < 5.0).astype(float)
        X['close'] = ((center_dist >= 5.0) & (center_dist < 15.0)).astype(float)
        X['medium'] = ((center_dist >= 15.0) & (center_dist < 30.0)).astype(float)
        X['far'] = (center_dist >= 30.0).astype(float)
    
    # LONG-RANGE TEMPORAL DEPENDENCIES FOR PAIRS
    time_windows = [5, 15, 30, 60]
    
    if 'body_center' in available_body_parts_A and 'body_center' in available_body_parts_B:
        # Inter-mouse distance over time
        center_dist_full = np.square(mouse_pair['A']['body_center'] - mouse_pair['B']['body_center']).sum(axis=1, skipna=False)
        
        for window in time_windows:
            # Distance statistics over windows
            X[f'dist_mean_{window}'] = center_dist_full.rolling(window, min_periods=1, center=True).mean()
            X[f'dist_std_{window}'] = center_dist_full.rolling(window, min_periods=1, center=True).std()
            X[f'dist_min_{window}'] = center_dist_full.rolling(window, min_periods=1, center=True).min()
            X[f'dist_max_{window}'] = center_dist_full.rolling(window, min_periods=1, center=True).max()
            
            # Interaction intensity (inverse of distance variance)
            dist_var = center_dist_full.rolling(window, min_periods=1, center=True).var()
            X[f'interaction_intensity_{window}'] = 1 / (1 + dist_var)
            
            # Coordinated movement over window
            A_x_diff = mouse_pair['A']['body_center']['x'].diff()
            A_y_diff = mouse_pair['A']['body_center']['y'].diff()
            B_x_diff = mouse_pair['B']['body_center']['x'].diff()
            B_y_diff = mouse_pair['B']['body_center']['y'].diff()
            
            coord_movement = A_x_diff * B_x_diff + A_y_diff * B_y_diff
            X[f'coord_movement_mean_{window}'] = coord_movement.rolling(window, min_periods=1, center=True).mean()
            X[f'coord_movement_std_{window}'] = coord_movement.rolling(window, min_periods=1, center=True).std()
    
    # Lag features for interaction patterns
    if 'nose' in available_body_parts_A and 'nose' in available_body_parts_B:
        nose_nose_dist = np.sqrt(
            (mouse_pair['A']['nose']['x'] - mouse_pair['B']['nose']['x'])**2 +
            (mouse_pair['A']['nose']['y'] - mouse_pair['B']['nose']['y'])**2
        )
        
        for lag in [10, 20, 40]:
            X[f'nose_nose_dist_lag_{lag}'] = nose_nose_dist.shift(lag)
            X[f'nose_nose_dist_change_{lag}'] = nose_nose_dist - nose_nose_dist.shift(lag)
            
            # Interaction persistence (how long they stay close)
            close_threshold = 10.0
            is_close = (nose_nose_dist < close_threshold).astype(float)
            X[f'close_persistence_{lag}'] = is_close.rolling(lag, min_periods=1).mean()
    
    # Temporal context for social behaviors
    if 'body_center' in available_body_parts_A and 'body_center' in available_body_parts_B:
        # Relative velocity alignment over time
        A_vel_x = mouse_pair['A']['body_center']['x'].diff()
        A_vel_y = mouse_pair['A']['body_center']['y'].diff()
        B_vel_x = mouse_pair['B']['body_center']['x'].diff()
        B_vel_y = mouse_pair['B']['body_center']['y'].diff()
        
        vel_alignment = (A_vel_x * B_vel_x + A_vel_y * B_vel_y) / (
            np.sqrt(A_vel_x**2 + A_vel_y**2) * np.sqrt(B_vel_x**2 + B_vel_y**2) + 1e-6
        )
        
        # Look at alignment patterns over different time horizons
        for offset in [-20, -10, 0, 10, 20]:
            X[f'vel_alignment_offset_{offset}'] = vel_alignment.shift(-offset)
        
        # Temporal consistency of interaction
        X['interaction_consistency_30'] = center_dist_full.rolling(30, min_periods=1, center=True).std() / (center_dist_full.rolling(30, min_periods=1, center=True).mean() + 1e-6)
    
    return X#*1.04

# Cross-validation function
def cross_validate_classifier(binary_classifier, X, label, meta):
    oof = pd.DataFrame(index=meta.video_frame)
    for action in label.columns:
        action_mask = ~ label[action].isna().values
        X_action = X[action_mask]
        y_action = label[action][action_mask].values.astype(int)
        p = y_action.mean()
        baseline_score = p / (1 + p)
        groups_action = meta.video_id[action_mask]
        if len(np.unique(groups_action)) < 5:
            continue

        if not (y_action == 0).all():
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore', category=RuntimeWarning)
                oof_action = cross_val_predict(binary_classifier, X_action, y_action, groups=groups_action, cv=GroupKFold(), method='predict_proba')
            oof_action = oof_action[:, 1]
        else:
            oof_action = np.zeros(len(y_action))
        f1 = f1_score(y_action, (oof_action >= threshold), zero_division=0)
        ch = '>' if f1 > baseline_score else '=' if f1 == baseline_score else '<'
        print(f"  F1: {f1:.3f} {ch} ({baseline_score:.3f}) {action}")
        f1_list.append((body_parts_tracked_str, action, f1))
        oof_column = np.zeros(len(label))
        oof_column[action_mask] = oof_action
        oof[action] = oof_column

    submission_part = predict_multiclass(oof, meta)
    submission_list.append(submission_part)

# Submit function
def submit(body_parts_tracked_str, switch_tr, binary_classifier, X_tr, label, meta):
    model_list = []
    for action in label.columns:
        action_mask = ~ label[action].isna().values
        y_action = label[action][action_mask].values.astype(int)

        if not (y_action == 0).all():
            model = clone(binary_classifier)
            model.fit(X_tr[action_mask], y_action)
            assert len(model.classes_) == 2
            model_list.append((action, model))

    body_parts_tracked = json.loads(body_parts_tracked_str)
    if len(body_parts_tracked) > 5:
        body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]
    if validate_or_submit == 'submit':
        test_subset = test[test.body_parts_tracked == body_parts_tracked_str]
        generator = generate_mouse_data(test_subset, 'test',
                                        generate_single=(switch_tr == 'single'), 
                                        generate_pair=(switch_tr == 'pair'))
    else:
        test_subset = stresstest.query("body_parts_tracked == @body_parts_tracked_str")
        generator = generate_mouse_data(test_subset, 'test',
                                        traintest_directory='stresstest_tracking',
                                        generate_single=(switch_tr == 'single'),
                                        generate_pair=(switch_tr == 'pair'))
    if verbose: print(f"n_videos: {len(test_subset)}")
    for switch_te, data_te, meta_te, actions_te in generator:
        assert switch_te == switch_tr
        try:
            if switch_te == 'single':
                X_te = transform_single(data_te, body_parts_tracked)
            else:
                X_te = transform_pair(data_te, body_parts_tracked)
            if verbose and len(X_te) == 0: print("ERROR: X_te is empty")
            del data_te
    
            pred = pd.DataFrame(index=meta_te.video_frame)
            for action, model in model_list:
                if action in actions_te:
                    pred[action] = model.predict_proba(X_te)[:, 1]
            del X_te
            if pred.shape[1] != 0:
                submission_part = predict_multiclass(pred, meta_te)
                submission_list.append(submission_part)
            else:
                if verbose: print(f"  ERROR: no useful training data")
        except KeyError:
            if verbose: print(f'  ERROR: KeyError because of missing bodypart ({switch_tr})')
            del data_te

# Robustify function
def robustify(submission, dataset, traintest, traintest_directory=None):
    if traintest_directory is None:
        traintest_directory = f"/kaggle/input/MABe-mouse-behavior-detection/{traintest}_tracking"

    old_submission = submission.copy()
    submission = submission[submission.start_frame < submission.stop_frame]
    if len(submission) != len(old_submission):
        print("ERROR: Dropped frames with start >= stop")
    
    old_submission = submission.copy()
    group_list = []
    for _, group in submission.groupby(['video_id', 'agent_id', 'target_id']):
        group = group.sort_values('start_frame')
        mask = np.ones(len(group), dtype=bool)
        last_stop_frame = 0
        for i, (_, row) in enumerate(group.iterrows()):
            if row['start_frame'] < last_stop_frame:
                mask[i] = False
            else:
                last_stop_frame = row['stop_frame']
        group_list.append(group[mask])
    submission = pd.concat(group_list)
    if len(submission) != len(old_submission):
        print("ERROR: Dropped duplicate frames")

    s_list = []
    for idx, row in dataset.iterrows():
        lab_id = row['lab_id']
        if lab_id.startswith('MABe22'):
            continue
        video_id = row['video_id']
        if (submission.video_id == video_id).any():
            continue

        if verbose: print(f"Video {video_id} has no predictions.")
        
        path = f"{traintest_directory}/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)
    
        vid_behaviors = eval(row['behaviors_labeled'])
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(',') for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])
    
        start_frame = vid.video_frame.min()
        stop_frame = vid.video_frame.max() + 1
    
        for (agent, target), actions in vid_behaviors.groupby(['agent', 'target']):
            batch_length = int(np.ceil((stop_frame - start_frame) / len(actions)))
            for i, (_, action_row) in enumerate(actions.iterrows()):
                batch_start = start_frame + i * batch_length
                batch_stop = min(batch_start + batch_length, stop_frame)
                s_list.append((video_id, agent, target, action_row['action'], batch_start, batch_stop))

    if len(s_list) > 0:
        submission = pd.concat([
            submission,
            pd.DataFrame(s_list, columns=['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])
        ])
        print("ERROR: Filled empty videos")

    submission = submission.reset_index(drop=True)
    return submission

# Main processing loop
f1_list = []
submission_list = []

for section in range(1, len(body_parts_tracked_list)):
    body_parts_tracked_str = body_parts_tracked_list[section]
    try:
        body_parts_tracked = json.loads(body_parts_tracked_str)
        print(f"{section}. Processing videos with {body_parts_tracked}")
        if len(body_parts_tracked) > 5:
            body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]
    
        train_subset = train[train.body_parts_tracked == body_parts_tracked_str]
        single_mouse_list = []
        single_mouse_label_list = []
        single_mouse_meta_list = []
        mouse_pair_list = []
        mouse_pair_label_list = []
        mouse_pair_meta_list = []
    
        for switch, data, meta, label in generate_mouse_data(train_subset, 'train'):
            if switch == 'single':
                single_mouse_list.append(data)
                single_mouse_meta_list.append(meta)
                single_mouse_label_list.append(label)
            else:
                mouse_pair_list.append(data)
                mouse_pair_meta_list.append(meta)
                mouse_pair_label_list.append(label)
    
        binary_classifier = make_pipeline(
            SimpleImputer(),
            TrainOnSubsetClassifier(
                lightgbm.LGBMClassifier(
                    n_estimators=200, 
                    learning_rate=0.06,
                    min_child_samples=40,
                    num_leaves=31,
                    max_depth=-1,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    verbose=-1),
                100000)
        )
    
        if len(single_mouse_list) > 0:
            single_mouse = pd.concat(single_mouse_list)
            single_mouse_label = pd.concat(single_mouse_label_list)
            single_mouse_meta = pd.concat(single_mouse_meta_list)
            del single_mouse_list, single_mouse_label_list, single_mouse_meta_list
            assert len(single_mouse) == len(single_mouse_label)
            assert len(single_mouse) == len(single_mouse_meta)
            
            X_tr = transform_single(single_mouse, body_parts_tracked)
            del single_mouse
            print(f"{X_tr.shape=}")
    
            if validate_or_submit == 'validate':
                cross_validate_classifier(binary_classifier, X_tr, single_mouse_label, single_mouse_meta)
            else:
                submit(body_parts_tracked_str, 'single', binary_classifier, X_tr, single_mouse_label, single_mouse_meta)
            del X_tr
                
        if len(mouse_pair_list) > 0:
            mouse_pair = pd.concat(mouse_pair_list)
            mouse_pair_label = pd.concat(mouse_pair_label_list)
            mouse_pair_meta = pd.concat(mouse_pair_meta_list)
            del mouse_pair_list, mouse_pair_label_list, mouse_pair_meta_list
            assert len(mouse_pair) == len(mouse_pair_label)
            assert len(mouse_pair) == len(mouse_pair_meta)
        
            X_tr = transform_pair(mouse_pair, body_parts_tracked)
            del mouse_pair
            print(f"{X_tr.shape=}")
    
            if validate_or_submit == 'validate':
                cross_validate_classifier(binary_classifier, X_tr, mouse_pair_label, mouse_pair_meta)
            else:
                submit(body_parts_tracked_str, 'pair', binary_classifier, X_tr, mouse_pair_label, mouse_pair_meta)
            del X_tr
                
    except Exception as e:
        print(f'***Exception*** {e}')
    print()

# Final submission creation
if validate_or_submit == 'validate':
    submission = pd.concat(submission_list)
    submission_robust = robustify(submission, train, 'train')
    print(f"# OOF score with competition metric: {score(solution, submission_robust, ''):.4f}")

    f1_df = pd.DataFrame(f1_list, columns=['body_parts_tracked_str', 'action', 'binary F1 score'])
    print(f"# Average of {len(f1_df)} binary F1 scores {f1_df['binary F1 score'].mean():.4f}")

if validate_or_submit != 'validate':
    if len(submission_list) > 0:
        submission = pd.concat(submission_list)
    else:
        submission = pd.DataFrame(
            dict(
                video_id=438887472,
                agent_id='mouse1',
                target_id='self',
                action='rear',
                start_frame='278',
                stop_frame='500'
            ), index=[44])
    if validate_or_submit == 'submit':
        submission_robust = robustify(submission, test, 'test')
    else:
        submission_robust = robustify(submission, stresstest, 'stresstest', 'stresstest_tracking')
    submission_robust.index.name = 'row_id'
    submission_robust.to_csv('submission.csv')
    print("Submission file created: submission.csv")